In [33]:
import torch
import requests
import os
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
from tqdm import tqdm
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [4]:
Noise_0_dataframe = pd.read_csv("Data/Assignment1/data_0_noise")
Noise_Low_dataframe = pd.read_csv("Data/Assignment1/data_Low_noise")
Noise_High_dataframe = pd.read_csv("Data/Assignment1/data_High_noise")

In [124]:
dataframe=Noise_Low_dataframe
validation_dataframe=Noise_Low_dataframe
target_columns="era"

In [125]:
to_encode = "era"
class_index = list(dataframe[to_encode].unique())
def encode(value, class_index = class_index):
    return class_index.index(value)

dataframe[to_encode] = dataframe[to_encode].apply(encode)
validation_dataframe[to_encode] = validation_dataframe[to_encode].apply(encode)

In [127]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, noise, transform=None, target_transform=None, drop=None, target=None, encoderdecoder=False):
        self.dataframe = dataframe
        if drop != None:
            self.X = dataframe.drop(drop, axis=1).values
        else:
            self.X = dataframe.values

        if not encoderdecoder:
            self.y = dataframe[target].values
        else:
            self.y = self.X
        self.transform = transform
        self.target_transform = target_transform
        self.noise = noise

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item, label = self.X[idx], self.y[idx]
        return item, label

    def get_noise(self):
        return self.noise

In [128]:
dataset = CustomDataset(dataframe, "0",drop = ["row_num","day","era","target_10_val","target_5_val","data_type"],target=target_columns)
Noise_train, Noise_test = random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
Noise_train_loader = DataLoader(Noise_train, batch_size=128, shuffle=True)
Noise_test_loader = DataLoader(Noise_test, batch_size=128, shuffle=True)

In [153]:
class MyMLP(torch.nn.Module):
    def __init__(self,ModuleList):
        super(MyMLP, self).__init__()
        module_list = torch.nn.ModuleList(ModuleList)
        self.layers = torch.nn.Sequential(*module_list)
        self.softmax = torch.nn.Softmax(dim=1)
    
    def forward(self,X):
        X = self.layers(X)
        X = self.softmax(X)
        return X

In [43]:
class SubTabEncoderDecoder(torch.nn.Module):
    def __init__(self,encoder_sizes,decoder_sizes,activation = torch.nn.ReLU()):
        super().__init__()
        self.encoder_sizes = encoder_sizes
        self.decoder_sizes = decoder_sizes
        linear_encoder = [torch.nn.Linear(encoder_sizes[i],encoder_sizes[i+1]) for i in range(len(encoder_sizes)-1)]
        linear_decoder = [torch.nn.Linear(decoder_sizes[i],decoder_sizes[i+1]) for i in range(len(decoder_sizes)-1)]
        self.encoder = torch.nn.Sequential(*[l for layer in linear_encoder for l in (layer, activation)])
        self.decoder = torch.nn.Sequential(*[l for layer in linear_decoder for l in (layer, activation)])
        
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def get_encoder(self):
        return self.encoder
    
    def get_decoder(self):
        return self.decoder

In [98]:
model = SubTabEncoderDecoder([24,32,16],[16,32,24])
model = model.to(device)

In [168]:
def train(model,criterion,optimizer,train_loader,val_loader,num_epochs = 100,verbos = True, lr = 0.001):
    optimizer = optimizer(model.parameters(), lr=lr)
    train_loss = []
    val_loss = []
    for epoch in range(num_epochs):
        model.train()
        current_train_loss = 0
        current_accuracy = []
        for data, target in tqdm(train_loader,desc = "Training Epoch "+str(epoch)):
            data, target = data.to(device).float(), target.to(device).long()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            current_train_loss += loss.item()
            current_accuracy.append(torch.sum(torch.argmax(output,dim=1) == target).item()/len(target))
            
        if (verbos):
            print("Epoch: ",epoch," Loss: ",current_train_loss/len(train_loader))
            print("Accuracy: ",np.mean(current_accuracy))
            
        train_loss.append(current_train_loss/len(train_loader))
        model.eval()
        current_val_loss = 0
        current_val_accuracy = []
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device).float(), target.to(device).long()
                output = model(data)
                loss = criterion(output, target)
                current_val_loss += loss.item()
                current_val_accuracy.append(torch.sum(torch.argmax(output,dim=1) == target).item()/len(target))
            val_loss.append(current_val_loss/len(val_loader))
            if (verbos):
                print("Validation Loss: ",current_val_loss/len(val_loader))
                print("Validation Accuracy: ",np.mean(current_val_accuracy))

In [114]:
encoder = model.get_encoder()
torch.save(encoder, "encoder.pth")

In [179]:
encoder = torch.load("encoder.pth")
for param in encoder.parameters():
    param.requires_grad = False
    print(param)

modules = [encoder]
modules.append(torch.nn.Linear(16, 16))
modules.append(torch.nn.Linear(16,12))
model = MyMLP(modules)
model = model.to(device)

Parameter containing:
tensor([[ 5.4385e-02,  8.8948e-02,  4.4324e-02,  1.4275e-01, -5.7704e-02,
         -2.7618e-01,  7.2716e-02,  8.1099e-02,  2.0728e-01,  2.6289e-01,
          4.9600e-01, -7.6064e-02, -2.2410e-03, -8.9613e-02,  1.0795e-01,
          1.6437e-01, -2.9043e-01,  1.4872e-01,  2.6488e-01,  9.9153e-02,
          4.2092e-01,  1.4136e-01,  4.8081e-02, -4.3162e-02],
        [-1.6995e-01,  4.2841e-02, -8.1688e-02, -6.8102e-02,  4.0787e-02,
          3.1971e-01,  3.5013e-01,  1.4828e-01,  3.4598e-01, -1.2330e-01,
         -3.8470e-02,  2.1789e-01, -1.3521e-01,  1.6085e-01, -9.4290e-02,
          3.1923e-01,  2.3472e-01,  9.9152e-02,  7.9975e-03,  4.6274e-01,
         -1.2566e-01, -5.0012e-02,  4.3919e-02,  4.3021e-01],
        [-1.0232e-01, -4.2814e-03,  2.1391e-01,  4.5100e-03, -1.0470e-01,
          2.6734e-02,  1.2554e-01,  2.1296e-01, -2.4479e-02, -1.5362e-01,
          1.4884e-02, -5.5587e-02,  1.8573e-01,  8.2411e-02,  1.6707e-01,
          5.8941e-02,  2.2441e-02,  2.67

In [178]:
train(model,torch.nn.CrossEntropyLoss(),torch.optim.Adam,Noise_train_loader,Noise_test_loader,num_epochs=100)

Training Epoch 0: 100%|██████████| 1950/1950 [00:05<00:00, 359.14it/s]


Epoch:  0  Loss:  2.3669971600556985
Accuracy:  0.2544471153846154
Validation Loss:  2.2934760622313766
Validation Accuracy:  0.3462634477459016


Training Epoch 1: 100%|██████████| 1950/1950 [00:04<00:00, 409.18it/s]


Epoch:  1  Loss:  2.270648723504482
Accuracy:  0.3690665064102564
Validation Loss:  2.2456896818075025
Validation Accuracy:  0.393890881147541


Training Epoch 2: 100%|██████████| 1950/1950 [00:04<00:00, 400.99it/s]


...

Epoch:  45  Loss:  2.172590602361239
Accuracy:  0.44774839743589745
Validation Loss:  2.1725079050806704
Validation Accuracy:  0.4471855788934426


Training Epoch 46: 100%|██████████| 1950/1950 [00:05<00:00, 380.27it/s]


Epoch:  46  Loss:  2.1725989708533655
Accuracy:  0.44756410256410256
Validation Loss:  2.1722887730012173
Validation Accuracy:  0.44854636270491804


Training Epoch 47:  72%|███████▏  | 1397/1950 [00:03<00:01, 393.19it/s]


KeyboardInterrupt: 